In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_ed66bba0165543c3bce9f08c268b209a.csv"
df=pd.read_csv(url)
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [2]:
df.shape

(517, 13)

In [3]:
df.dtypes

X          int64
Y          int64
month     object
day       object
FFMC     float64
DMC      float64
DC       float64
ISI      float64
temp     float64
RH         int64
wind     float64
rain     float64
area     float64
dtype: object

In [4]:
objects=df[["month","day"]]

In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
objects=objects.apply(le.fit_transform)
objects.head()

,month,day
0,7,0
1,10,5
2,10,2
3,7,0
4,7,3


In [6]:
sub_df=df.drop(["month","day"],axis=1)
df=pd.concat([objects,sub_df],axis=1)
df.head()

,month,day,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,10,5,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,10,2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,7,0,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,7,3,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [7]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(random_state=0).fit(df)
outlier_label=pd.DataFrame(clf.predict(df))
outlier_label=outlier_label.rename(columns={0:"outlier_label"})
outlier_label

,outlier_label
0,1
1,1
2,1
3,-1
4,-1
...,...
512,1
513,1
514,1
515,1


In [8]:
len(outlier_label[outlier_label["outlier_label"]==-1])

61

In [9]:
df_labeled=pd.concat([df,outlier_label],axis=1)
outliers=df_labeled[df_labeled["outlier_label"]==-1].drop(["outlier_label"],axis=1)
outliers

,month,day,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
3,7,0,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,7,3,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
12,1,0,6,5,63.5,70.8,665.3,0.8,17.0,72,6.7,0.0,0.00
58,3,1,2,2,84.0,9.3,34.0,2.1,13.9,40,5.4,0.0,0.00
59,3,0,2,2,86.6,13.2,43.0,5.3,12.3,51,0.9,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,1,5,8,6,96.1,181.1,671.2,14.3,21.6,65,4.9,0.8,0.00
501,1,5,7,5,96.1,181.1,671.2,14.3,21.6,65,4.9,0.8,0.00
506,1,0,1,2,91.0,166.9,752.6,7.1,18.5,73,8.5,0.0,0.00
509,1,0,5,4,91.0,166.9,752.6,7.1,21.1,71,7.6,1.4,2.17


In [10]:
df_labeled=pd.concat([df,outlier_label],axis=1)
non_outliers=df_labeled[df_labeled["outlier_label"]==1].drop(["outlier_label"],axis=1)
non_outliers

,month,day,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,10,5,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,10,2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
5,1,3,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.00
6,1,1,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,1,3,8,6,81.6,56.7,665.6,1.9,27.8,35,2.7,0.0,0.00
512,1,3,4,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,1,3,2,4,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,1,3,7,4,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16


In [11]:
X=non_outliers.drop(["X","Y","area"],axis=1)
y=non_outliers[["area"]]

In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,0.249673,-1.430301,-1.950064,-1.465693,-2.074346,-0.928047,-2.105856,0.590431,1.694368,-0.046881
1,0.926013,1.203856,-0.299244,-1.310383,0.428366,-0.567982,-0.264206,-0.669319,-1.793812,-0.046881
2,0.926013,-0.376638,-0.299244,-1.170267,0.505868,-0.567982,-0.903145,-0.669319,-1.553248,-0.046881
3,-1.103009,0.150193,0.338573,-0.467998,-0.360154,1.232346,0.525073,-0.949263,0.912535,-0.046881
4,-1.103009,-0.903470,0.338573,-0.407225,-0.327063,-0.162908,0.882128,-1.089236,-0.470709,-0.046881


In [13]:
y=pd.DataFrame(scaler.fit_transform(y),columns=y.columns)
y.head()

,area
0,-0.353539
1,-0.353539
2,-0.353539
3,-0.353539
4,-0.353539


In [14]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


test = SelectKBest(score_func=f_regression, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
6,1.829902
8,1.516971
0,0.508026
3,0.494175
5,0.288097
7,0.168032
9,0.124749
1,0.045934
4,0.037684
2,0.020617


In [15]:
X_scores=scores[scores["scores"]>0.1]
len(X_scores)

7

In [16]:
X=X.iloc[:,X_scores.index]
X.head()

,temp,wind,month,DMC,ISI,RH,rain
0,-2.105856,1.694368,0.249673,-1.465693,-0.928047,0.590431,-0.046881
1,-0.264206,-1.793812,0.926013,-1.310383,-0.567982,-0.669319,-0.046881
2,-0.903145,-1.553248,0.926013,-1.170267,-0.567982,-0.669319,-0.046881
3,0.525073,0.912535,-1.103009,-0.467998,1.232346,-0.949263,-0.046881
4,0.882128,-0.470709,-1.103009,-0.407225,-0.162908,-1.089236,-0.046881


In [17]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [18]:
Gradient=GradientBoostingRegressor(loss="lad",learning_rate=0.01,criterion="mse",n_estimators=250,random_state=0)
scores = cross_val_score(Gradient, X, np.asarray(y).reshape(y.shape[0],), scoring='neg_mean_squared_error', cv=50)
print("Negative mean squared error: ",scores.mean(),"+/-",scores.std())

Negative mean squared error:  -1.120672983648584 +/- 3.2667550654590944


In [19]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(Gradient, X, np.asarray(y).reshape(y.shape[0],), cv=50)
y_pred =pd.DataFrame(scaler.inverse_transform(y_pred)).rename(columns={0:"Predicted burn area"})
print("Mean absolute deviation for predicted burn area: ",y_pred.mad())

Mean absolute deviation for predicted burn area:  Predicted burn area    0.384666
dtype: float64


In [20]:
from sklearn.metrics import mean_squared_error as mse
y=pd.DataFrame(scaler.inverse_transform(y),columns=y.columns)
print("Root mean squared error: ",np.sqrt(mse(y,y_pred)))

Root mean squared error:  23.639981601666683


Build a separate model for the outliers.

In [21]:
X_o=outliers.drop(["X","Y","area"],axis=1)
y_o=outliers[["area"]]

In [22]:
scaler_o=StandardScaler()
X_o=pd.DataFrame(scaler_o.fit_transform(X_o),columns=X_o.columns)
X_o.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,0.609847,-1.385861,0.514521,-0.680191,-0.982440,0.294215,-0.893922,1.717297,-0.444408,0.027219
1,0.609847,0.046978,0.329763,-0.481106,-0.901599,0.412678,-0.481929,1.803551,-1.397117,-0.209978
2,-1.019380,-1.385861,-1.656386,-0.265430,0.941373,-1.324777,0.262316,0.639126,0.724825,-0.209978
3,-0.476304,-0.908248,-0.078245,-0.945639,-1.124812,-1.068107,-0.149677,-0.740933,0.161861,-0.209978
4,-0.476304,-1.385861,0.121910,-0.902504,-1.095355,-0.436306,-0.362318,-0.266538,-1.786862,-0.209978


In [23]:
y_o=pd.DataFrame(scaler_o.fit_transform(y_o),columns=y_o.columns)
y_o.head()

,area
0,-0.290652
1,-0.290652
2,-0.290652
3,-0.290652
4,-0.290652


In [24]:
test = SelectKBest(score_func=f_regression, k="all")
fit = test.fit(X_o, np.asarray(y_o).reshape(y_o.shape[0],))
features = fit.transform(X_o)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
6,6.724261
7,5.501491
4,4.138467
3,2.105065
2,1.570094
0,1.239032
5,0.835211
8,0.823842
9,0.164784
1,0.042476


In [25]:
X_oscore=scores[scores["scores"]>1]
len(X_oscore)

6

In [26]:
Gradient_o=GradientBoostingRegressor(loss="lad",learning_rate=0.03,criterion="mse",n_estimators=200,random_state=0)
scores = cross_val_score(Gradient_o, X_o, np.asarray(y_o).reshape(y_o.shape[0],), scoring='neg_mean_squared_error', cv=35)
print("Negative mean squared error: ",scores.mean(),"+/-",scores.std())

Negative mean squared error:  -0.9272467729298981 +/- 3.7508587576181847


In [27]:
y_opred = cross_val_predict(Gradient_o, X_o, np.asarray(y_o).reshape(y_o.shape[0],), cv=35)
y_opred =pd.DataFrame(scaler_o.inverse_transform(y_opred)).rename(columns={0:"Predicted burn area"})
print("Mean absolute deviation for predicted burn area: ",y_opred.mad())

Mean absolute deviation for predicted burn area:  Predicted burn area    18.350683
dtype: float64


In [29]:
y_o=pd.DataFrame(scaler_o.inverse_transform(y_o),columns=y_o.columns)
print("Root mean squared error for the outliers: ",np.sqrt(mse(y_o,y_opred)))

Root mean squared error for the outliers:  175.41144735623405


What if I did not implement an outlier detection algorithm?

In [93]:
X=df.drop(["X","Y","area"],axis=1)
y=df[["area"]]

In [94]:
scaler2=StandardScaler()
X=pd.DataFrame(scaler2.fit_transform(X),columns=X.columns)
X.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,0.284222,-1.423121,-0.805959,-1.323326,-1.830477,-0.860946,-1.842640,0.411724,1.498614,-0.073268
1,0.970871,1.176715,-0.008102,-1.179541,0.488891,-0.509688,-0.153278,-0.692456,-1.741756,-0.073268
2,0.970871,-0.383187,-0.008102,-1.049822,0.560715,-0.509688,-0.739383,-0.692456,-1.518282,-0.073268
3,0.284222,-1.423121,0.191362,-1.212361,-1.898266,-0.004756,-1.825402,3.233519,-0.009834,0.603155
4,0.284222,0.136781,-0.243833,-0.931043,-1.798600,0.126966,-1.291012,3.356206,-1.238940,-0.073268


In [95]:
y=pd.DataFrame(scaler2.fit_transform(y),columns=y.columns)
y.head()

,area
0,-0.20202
1,-0.20202
2,-0.20202
3,-0.20202
4,-0.20202


In [96]:
test = SelectKBest(score_func=f_regression, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
6,4.977993
7,2.953919
3,2.758705
4,1.259002
2,0.830371
0,0.497117
1,0.184337
8,0.078145
5,0.035120
9,0.027942


In [97]:
X_scores=scores[scores["scores"]>0.1]
len(X_scores)

7

In [98]:
X=X.iloc[:,X_scores.index]
X.head()

,temp,RH,DMC,DC,FFMC,month,day
0,-1.842640,0.411724,-1.323326,-1.830477,-0.805959,0.284222,-1.423121
1,-0.153278,-0.692456,-1.179541,0.488891,-0.008102,0.970871,1.176715
2,-0.739383,-0.692456,-1.049822,0.560715,-0.008102,0.970871,-0.383187
3,-1.825402,3.233519,-1.212361,-1.898266,0.191362,0.284222,-1.423121
4,-1.291012,3.356206,-0.931043,-1.798600,-0.243833,0.284222,0.136781


In [99]:
Gradient2=GradientBoostingRegressor(loss="lad",learning_rate=0.01,criterion="mse",n_estimators=250,random_state=0)
scores = cross_val_score(Gradient2, X, np.asarray(y).reshape(y.shape[0],), scoring='neg_mean_squared_error', cv=50)
print("Negative mean squared error: ",scores.mean(),"+/-",scores.std())

Negative mean squared error:  -1.0707923712854304 +/- 4.648424317831928


In [100]:
y_pred = cross_val_predict(Gradient2, X, np.asarray(y).reshape(y.shape[0],), cv=50)
y_pred =pd.DataFrame(scaler2.inverse_transform(y_pred)).rename(columns={0:"Predicted burn area"})
print("Mean absolute deviation for predicted burn area: ",y_pred.mad())

Mean absolute deviation for predicted burn area:  Predicted burn area    0.552748
dtype: float64


Filtering out the outliers lowered the mean absolute deviation for the predicted burn area.

In [101]:
from sklearn.metrics import mean_squared_error as mse
y=pd.DataFrame(scaler2.inverse_transform(y),columns=y.columns)
print("Root mean squared error : ",np.sqrt(mse(y,y_pred)))

Root mean squared error :  64.71650457705749


### Summary

|Model|Mean absolute deviation (MAD)|Root mean squared error (RMSE)|
|-----|-----------------------------|------------------------------|
|Without the outliers|0.384666|23.639981601666683|
|Just the outliers|18.350683|175.41144735623405|
|With the outliers|0.552748|64.71650457705749|